# Input Data and Classification Model

### The Milky Way was so meh! Been there, done that!


Every group of document are separate by language and label. I am going to unify the languages to make three different models, one per language. So, the input must be in the same language and with the specific label.

In [1]:
# Imports
import pandas as pd
import numpy as np
import re
import glob, os

import spacy

import string

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

In [2]:
from spacy.lang.en.stop_words import STOP_WORDS
stopwords_en = list(STOP_WORDS)
from spacy.lang.fr.stop_words import STOP_WORDS
stopwords_fr = list(STOP_WORDS)
from spacy.lang.es.stop_words import STOP_WORDS
stopwords_es = list(STOP_WORDS)

from spacy.lang.en import English
from spacy.lang.fr import French
from spacy.lang.es import Spanish


# 0. Preparing Data
In this part we are going to read the documents in the folders and join then in the same dataframe with the different labels for the training model. Because we are going to create 3 different models -one for each language- we are going to separate the dataframes because of the language.

In [3]:
def union_df(path, label):
    df_all = pd.DataFrame()
    for f in glob.glob(path + '/*.txt'):
        doc = [open(f, encoding='utf-8').read()]
        df = pd.DataFrame(doc, columns=['text'])
        df['label'] = label
        #df['file_name'] = re.findall('.+\/(.+\.txt)', f)
        #df = df[['label', 'text']]
        df_all = pd.concat([df_all,df], ignore_index = True)
    return df_all

In [4]:
label1 = 'APR'
label2 = 'Conference_papers'
label3 = 'PAN11'
label4 = 'Wikipedia'

# 1. English

### 1.1. Joing Dataframes

In [5]:
path1 = '/Users/Natalio/Desktop/nlp_associate_ds_test/NLP_Associate_DS_Test/data/documents_challenge/APR/en'

path2 = '/Users/Natalio/Desktop/nlp_associate_ds_test/NLP_Associate_DS_Test/data/documents_challenge/Conference_papers/en'

path3 = '/Users/Natalio/Desktop/nlp_associate_ds_test/NLP_Associate_DS_Test/data/documents_challenge/PAN11/en'

path4 = '/Users/Natalio/Desktop/nlp_associate_ds_test/NLP_Associate_DS_Test/data/documents_challenge/Wikipedia/en'


In [6]:
df_en1 = union_df(path1,label1)


In [7]:
df_en2 = union_df(path2,label2)


In [8]:
df_en3 = union_df(path3,label3)


In [9]:
df_en4 = union_df(path4,label4)

In [10]:
df_en = pd.concat([df_en1,df_en2, df_en3, df_en4], ignore_index = True)
df_en

,text,label
0,"i read this book because in my town, everyone ...",APR
1,recipes appreciated by the family (small and l...,APR
2,i say no to ease ..... and not to the author w...,APR
3,milady has found a good vein: anita blake. bas...,APR
4,"460 bc, somewhere in greece: ""gentlemen, i dec...",APR
...,...,...
9644,"Bupyeong-gu, Incheon. | location_country =...",Wikipedia
9645,Freedom Call is a German power metal band form...,Wikipedia
9646,majesty|consortname = Paola Ruffo di Calabriat...,Wikipedia
9647,Sertã (pron. ) is a municipality in Portugal ...,Wikipedia


I will be check few things about the new English dataframe:
- Lenght
- Columns
- Nulls

In [11]:
def checkingdf(df):
    length = len(df)
    columns = df.columns
    nulls = df.isnull().sum()
    return length, columns, nulls

In [12]:
checkingdf(df_en)

(9649,
 Index(['text', 'label'], dtype='object'),
 text     0
 label    0
 dtype: int64)

In [13]:
df_en.to_csv('/Users/Natalio/Desktop/nlp_associate_ds_test/NLP_Associate_DS_Test/data/training_data/english.csv', sep='\t', index=False)

Everything looks fine, or I should say... *Roger, Roger!*

### 1.2. Removing StopWords, Lemmanization, Tokenization and remove puntuation
Removing **stopwords** is an important step in NLP. It involves filtering out hihg-frecuency words that add little or no semantic value to a sentence.

**Lemmanization** is the act that convert the several infected forms of a word into a single form to make the analysis process easier. For example, if we have *I'am*, *you're* or *she wasn't* we will get the verb *to be* instead of the variations.

**Tokenization** is a technique where we divide the words into morphemes (basic unit of meaning).

Also, we are going to **remove puntuation** to reduce the content of the text files into words.

I am using SpaCy

In [14]:
lang = 'en'
nlp = spacy.load('en_core_web_md')

In [15]:
punctuations = string.punctuation
parser_en = English()
parser = parser_en

In [16]:
def spacy_tokenizer(sentence):
    #if lang == 'en':
     #   nlp = spacy.load(lang)
     #   tokens = English(sentence)
    #elif lang == 'fr':
     #   tokens == French(sentence)
    #elif lang == 'es':
     #   tokens == Spanish(sentence)
    #else:
     #   raise error
    tokens = parser(sentence)
    tokens = [w.lemma_.lower().strip() if w.lemma_ != '-PRON-!' else w.lower_ for w in tokens]
    tokens = [w for w in tokens if w not in stopwords_en and w not in punctuations]
    return tokens

### 1.3. ML with SKLearn

Custom transformer


In [18]:
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        return[clean_text(text) for text in X]
    def fit(self, X, y=None, **fit_params):
        return self
    def get_params(self, deep=True):
        return[]

def clean_text(text):
    return text.strip().lower()

In [19]:
# Vectorization
vectorizer = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
classifier = LinearSVC()

# Using Tfidf
tfvectorizer = TfidfVectorizer(tokenizer = spacy_tokenizer)

I am going to split the dataframe. We areg going to need to select the right columns of the dataframe, but also, we want to keep some data out from training model to test it. I am going to do a 80 - 20.

In [20]:
# Features
X = df_en['text']

# Classification Labels
y = df_en['label']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

In [22]:
# Apply the pipeline to clean, tokenize, vectorize and classify
pipe = Pipeline([('cleaner', predictors()),('vectorizer', vectorizer), ('classifier', classifier)])

In [23]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('cleaner', <__main__.predictors object at 0x1a241d1290>),
                ('vectorizer',
                 CountVectorizer(tokenizer=<function spacy_tokenizer at 0x1a2edcbcb0>)),
                ('classifier', LinearSVC())])

In [24]:
sample_prediction = pipe.predict(X_test)

In [25]:

for(sample, pred) in zip(X_test,sample_prediction):
    print(f'Predict: {pred}')

Predict: APR
Predict: Wikipedia
Predict: APR
Predict: APR
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Conference_papers
Predict: Wikipedia
Predict: APR
Predict: Wikipedia
Predict: APR
Predict: APR
Predict: Wikipedia
Predict: Wikipedia
Predict: APR
Predict: Wikipedia
Predict: Conference_papers
Predict: Wikipedia
Predict: PAN11
Predict: Wikipedia
Predict: Wikipedia
Predict: APR
Predict: Wikipedia
Predict: APR
Predict: APR
Predict: APR
Predict: Wikipedia
Predict: Wikipedia
Predict: APR
Predict: APR
Predict: APR
Predict: APR
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: APR
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: PAN11
Predict: PAN11
Predict: APR
Predict: APR
Predict: APR
Predict: PAN11
Predict: Wikipedia
Predict: PAN11
Predict: Wikipedia
Predict: Conference_papers
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: APR
Predict: PAN11
Predict: Wikipedia

In [32]:
print(f'Accuracy test:{pipe.score(X_test, y_test)}')
print(f'Accuracy sample:{pipe.score(X_test, sample_prediction)}')

Accuracy test:0.9569948186528497
Accuracy sample:1.0


In [33]:
print(f'Accuracy score:{pipe.score(X_train, y_train)}')

Accuracy score:1.0


In [38]:
pipe.predict(['With a scientific method we can show the data in this graph. We want to show our investigation results of five years with machine learning models'])

array(['APR'], dtype=object)

In [39]:
pipe.predict(['I do not like this item at all'])

array(['APR'], dtype=object)

In [40]:
pipe.predict(['and the computer that ran the first public Bulletin Board Systems, CBBS]]A Bulletin Board System, or BBS, is a computer system running software that allows users to connect and login to the system using a terminal program. Originally BBSes were accessed only over a phone line using a modem, but by the early 1990s some BBSes allowed access via a Telnet or packet radio connection.Once a user logged in, they could perform functions such as downloading or uploading software and data, reading news, and exchanging messages with other users. Many BBSes also offered on-line games, in which users could compete with each other, and BBSes with multiple phone lines often offered IRC-like chat rooms, allowing users to meet each other.In recent years, the term BBS is sometimes incorrectly used to refer to any online forum or message board.During their heyday from the late 1970s to the mid 1990s, most BBSes were run as a hobby free of charge by the system operator (or "sysop"), while other BBSes charged their users a subscription fee for access, or were operated by a business as a means of supporting their customers. Bulletin Board Systems were in many ways a precursor to the modern form of the World Wide Web and other aspects of the Internet.Early BBSes were often a local phenomenon, as one had to dial into a BBS'])

array(['Wikipedia'], dtype=object)

In [41]:
pipe.predict(['A Long Island iced tea is a type of alcoholic mixed drink typically made with vodka, tequila, light rum, triple sec, gin, and a splash of cola, which gives the drink the same amber hue as its namesake.[1] A popular version mixes equal parts vodka, tequila, gin, rum, triple sec, with ​1 1⁄2 parts sour mix and a splash of cola. Lastly, it is decorated with the lemon and straw, after stirring with bar spoon smoothly.[2]Most variants use equal parts of the main liquors, but include a smaller amount of triple sec (or other orange-flavored liqueur). Close variants often replace the sour mix with lemon juice, replace the cola with diet cola or actual iced tea, or add white crème de menthe. Most variants do not include any tea.The drink has a much higher alcohol concentration (approximately 22 percent) than most highball drinks due to the relatively small amount of mixer.'])

array(['APR'], dtype=object)

# 2. French

In [42]:
path1 = '/Users/Natalio/Desktop/nlp_associate_ds_test/NLP_Associate_DS_Test/data/documents_challenge/APR/fr'

path2 = '/Users/Natalio/Desktop/nlp_associate_ds_test/NLP_Associate_DS_Test/data/documents_challenge/Conference_papers/fr'

path4 = '/Users/Natalio/Desktop/nlp_associate_ds_test/NLP_Associate_DS_Test/data/documents_challenge/Wikipedia/fr'

In [43]:
df_fr1 = union_df(path1,label1)
df_fr2 = union_df(path2,label2)
df_fr4 = union_df(path4,label4)
df_fr = pd.concat([df_fr1, df_fr2, df_fr4], ignore_index = True)
df_fr

,text,label
0,"J'avais beaucoup aimé les premiers albums du ""...",APR
1,Je me joins aux commentaires peu satisfaits......,APR
2,"À sa parution en 1979, ce livre n'a pas rencon...",APR
3,Je découvre Douglas Kennedy et j'aimerais que ...,APR
4,J'ai acheté ce livre à la lecture des commenta...,APR
...,...,...
7958,", Nuremberg |années actives = depuis 1998 |gen...",Wikipedia
7959,Une cellule polyploïde (du grec : πολλαπλόν - ...,Wikipedia
7960,", George W. Bush et Albert II le .]]La reine ...",Wikipedia
7961,Sertã est une petite ville portugaise de 5 50...,Wikipedia


In [44]:
checkingdf(df_fr)

(7963,
 Index(['text', 'label'], dtype='object'),
 text     0
 label    0
 dtype: int64)

In [45]:
df_fr.to_csv('/Users/Natalio/Desktop/nlp_associate_ds_test/NLP_Associate_DS_Test/data/training_data/french.csv', header= False, index=False)

In [46]:
lang = 'fr'
nlp = spacy.load('fr_core_news_lg')
parser_fr = French()
parser = parser_fr
# Vectorization
vectorizer = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
classifier = LinearSVC()

# Using Tfidf
tfvectorizer = TfidfVectorizer(tokenizer = spacy_tokenizer)

# Features
X = df_fr['text']

# Classification Labels
y = df_fr['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

# Apply the pipeline to clean, tokenize, vectorize and classify
pipe = Pipeline([('cleaner', predictors()),('vectorizer', vectorizer), ('classifier', classifier)])

pipe.fit(X_train,y_train)

sample_prediction = pipe.predict(X_test)

/Users/Natalio/miniconda3/envs/ea_env/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [47]:

for(sample, pred) in zip(X_test,sample_prediction):
    print(f'Predict: {pred}')
print(f'Accuracy test:{pipe.score(X_test, y_test)}')
print(f'Accuracy sample:{pipe.score(X_test, sample_prediction)}')

Predict: APR
Predict: APR
Predict: Wikipedia
Predict: APR
Predict: Wikipedia
Predict: APR
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: APR
Predict: APR
Predict: APR
Predict: APR
Predict: APR
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: APR
Predict: Wikipedia
Predict: Wikipedia
Predict: APR
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: APR
Predict: Wikipedia
Predict: Wikipedia
Predict: APR
Predict: APR
Predict: APR
Predict: APR
Predict: Wikipedia
Predict: APR
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Conference_papers
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: APR
Predict: APR
Predict: APR
Predict: W

Accuracy test:0.9943502824858758
Accuracy sample:1.0


In [50]:
pipe.predict(["Le Long Island iced tea ou LIIT ou Long Island littéralement thé glacé Long Island en anglais est un cocktail à base de tequila gin vodka rhum liqueur doranges et cola Ce cocktail officiel de lIBA tient son nom de sa ressemblance à du thé glacé bien quil nen contienne pas)"])

array(['APR'], dtype=object)

# 3. Spanish

In [12]:
path3 = '/Users/Natalio/Desktop/nlp_associate_ds_test/NLP_Associate_DS_Test/data/documents_challenge/PAN11/es'

path4 = '/Users/Natalio/Desktop/nlp_associate_ds_test/NLP_Associate_DS_Test/data/documents_challenge/Wikipedia/es'

In [13]:
df_es3 = union_df(path3,label3)
df_es4 = union_df(path4,label4)
df_es = pd.concat([df_es3, df_es4], ignore_index = True)
df_es

,text,label
0,"El primero dia pafamos aquellas Lagunas, i pa...",PAN11
1,"A la puesta del Sol, por vnos llanos, i entre...",PAN11
2,"\n\nLa Tierra, por la maior parte, defde donde...",PAN11
3,\n\nCAP. XXXVI. De como hecimos hacer Iglesias...,PAN11
4,¡Asombra el imaginar lo que hubiera dado este...,PAN11
...,...,...
4992,| fecha_de_fallecimiento = 8 de enero de 1981|...,Wikipedia
4993,Red Hat es la compañía responsable de la creac...,Wikipedia
4994,Bashkortostán (en ruso: Республика Башкортоста...,Wikipedia
4995,|zona=Polinesia |hablantes=165.000 (censo 200...,Wikipedia


In [14]:
checkingdf(df_es)

(4997,
 Index(['text', 'label'], dtype='object'),
 text     0
 label    0
 dtype: int64)

In [15]:
df_es.to_csv('/Users/Natalio/Desktop/nlp_associate_ds_test/NLP_Associate_DS_Test/data/training_data/spanish.csv', header= False, index=False)

In [20]:
lang = 'es'
nlp = spacy.load('es_core_news_lg')
parser_en = Spanish()
parser = parser_en
# Vectorization
vectorizer = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
classifier = LinearSVC()

# Using Tfidf
tfvectorizer = TfidfVectorizer(tokenizer = spacy_tokenizer)

# Features
X = df_es['text']

# Classification Labels
y = df_es['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

# Apply the pipeline to clean, tokenize, vectorize and classify
pipe = Pipeline([('cleaner', predictors()),('vectorizer', vectorizer), ('classifier', classifier)])

pipe.fit(X_train,y_train)

sample_prediction = pipe.predict(X_test)

In [21]:
for(sample, pred) in zip(X_test,sample_prediction):
    print(f'Predict: {pred}')
print(f'Accuracy test:{pipe.score(X_test, y_test)}')
print(f'Accuracy sample:{pipe.score(X_test, sample_prediction)}')

Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: PAN11
Predict: Wikipedia
Predict: Wikipedia
Predict: PAN11
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: PAN11
Predict: PAN11
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: PAN11
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: PAN11
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: PAN11
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia
Predict: PAN11
Predict: Wikipedia
Predict: PAN11
Predict: Wikipedia
Predict: Wikipedia
Predict: PAN11
Predict: Wikipedia
Predict: PAN11
Predict: Wikipedia
Predict: Wikipedia
Predict: Wikipedia

Accuracy test:0.998
Accuracy sample:1.0


In [22]:
pipe.predict(['Un Té Helado Long Island es un tipo de bebida alcohólica mezclada hecha típicamente con, entre otros,vodka, ron blanco, triple seco, y ginebra.1​ Se llama así debido a su parecido con el color y el sabor del Té helado (Camellia sinensis). Una versión popular mezcla partes iguales de vodka, ginebra,ron, y triple seco con 1½ partes de sour mix y un toque de refresco de cola, lo cual le da a la bebida el mismo color ámbar de su fundador. La mayoría de las variantes usan partes iguales de los licores principales, pero incluye una parte pequeña de triple seco (u otro licor sabor naranja). Variantes cercanas reemplazan seguido el sour mix con jugo de limón, el refresco de cola con uno de cola de dieta o té helado, o agregan crema de menta; sin embargo, la mayoría de las variantes no incluyen té. Algunos restaurantes substituyen brandy por el tequila. Algunas variantes de la bebida tienen nombres alternativos como lo son Texas Iced Tea, Georgia Iced Tea, Tokyo Tea, Three Mile Island, y Adios Mother Fucker. La bebida tiene una más alta concentración de alcohol (aproximadamente 22 por ciento) que la mayoría de los cocteles debido a la relativamente pequeña porción de soda. El Long Island puede ser ordenado extra largo, con un mayor incremento de alcohol en la mezcla.'])

array(['PAN11'], dtype=object)

## 4. Conclusions
Model have a good ratio, but the model is overfit to APR label in all languages. It will be necessary to come back and keep cleaning data for a better result.


## 5. Next steps
With the objetive of improve the model it will be interesting to try:
- Balance the dataset. Equitative number from different labels
- Join the model with the function of language detection. With this step first we will check the language of the text and after the text come into the right model
- Check numbers in input data

# Topic Modeling

## Get ready; today is not gonna be easy

The objetive 2 of the test ask for a topic modeling. We are going to work to find the topics that are present in the different files.

In this notebook we will be using **Latent Dirchlet Allocation(LDA)** which is one of many topic modelling techniques designed specifically for text data.

Once the topic modeling technique is applied, we need to interpret the results and modify the parameters to get the best result.

In [142]:
df = df_en.groupby(['label'])
df

# Imports
import pandas as pd
from gensim import matutils, models
import scipy.sparse
from sklearn.feature_extraction.text import CountVectorizer

### 1. Topic Modeling 1

In [194]:
def union_df2(path, label):
    df_all = pd.DataFrame()
    for f in glob.glob(path + '/*.txt'):
        doc = [open(f, encoding='utf-8').read()]
        doc_all.append(doc)
    combined_text = ' '.join([str(elem) for elem in doc_all])
    dic = {label:combined_text}
    #data_combined = {key: [combine_text(value)] for (key, value) in data.items()}
    #df = pd.DataFrame(combined_text)
    #df['label'] = label
        #df['file_name'] = re.findall('.+\/(.+\.txt)', f)
        #df = df[['label', 'text']]
        #df_all = pd.concat([df_all,df], ignore_index = True)
    return dic


In [200]:
a = union_df2(path1, label1)
b = union_df2(path2,label2)
c = union_df2(path3, label3)
d = union_df2(path4, label4)
a.update(b)
a.update(c)
a.update(d)
a

In [ ]:
data_df = pd.DataFrame.from_dict(a).transpose()
#data_df.columns = ['transcript']
#data_df = data_df.sort_index()
data_df

Text is too long for spaCy library, so we are not going to use "spacy_tokenizer" function in this case

Error: ValueError: [E088] Text of length 1947606 exceeds maximum of 1000000. The v2.x parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`.
        
    NOTE: Do not working text new limit

In [113]:
stopwords_en

['because',
 'cannot',
 'only',
 'below',
 'twenty',
 'same',
 'most',
 'alone',
 'of',
 "'re",
 'seemed',
 'never',
 'latter',
 'latterly',
 'back',
 'formerly',
 'anywhere',
 'hence',
 "n't",
 're',
 'although',
 '‘ll',
 'besides',
 'still',
 'mostly',
 'were',
 'off',
 'both',
 'otherwise',
 'myself',
 'out',
 'anyway',
 'made',
 'meanwhile',
 'why',
 'may',
 'former',
 'up',
 'she',
 'nevertheless',
 'by',
 'behind',
 'enough',
 'also',
 'more',
 'how',
 'being',
 'it',
 'side',
 'sometimes',
 'that',
 'nobody',
 'whose',
 '‘s',
 'twelve',
 'nothing',
 'move',
 'must',
 'indeed',
 'anyhow',
 'really',
 'somewhere',
 'anything',
 'none',
 'is',
 'serious',
 'what',
 'doing',
 'against',
 'thereafter',
 'thru',
 'five',
 'all',
 'now',
 'does',
 '‘m',
 'becomes',
 'another',
 'and',
 '’d',
 'thereby',
 'are',
 'whole',
 "'s",
 'ever',
 'would',
 'can',
 'unless',
 'therefore',
 'sixty',
 'whether',
 '‘ve',
 'become',
 'due',
 'has',
 'but',
 'where',
 '‘d',
 'other',
 'beforehand',
 

In [70]:
a

'["i read this book because in my town, everyone uses it and order. this is my pharmacist who advised me she was so thin i asked her what she had done and instead of just selling snake oil capsules, she advised me this book to 5 euros. of course, we must make an effort to lose 25 pounds but with the book, i had a companion. the author was able to talk to me just with strong arguments and above all i felt he knew many cases like mine. he is in his full experience, simplicity and compassion for those like me who lived with all that weight that stuck to my body and never want to leave. i do not think it is a fad diet that outperforms the others but i do believe that there are people who can speak to others and to be born of clicks. i might be low but this book made me strong, i have annotated so that i\'m on my third. when one is very big as i was, non-large do not understand or are afraid to offend you by speaking, then this book was like a companion journal. i am a pedicure and i have a

In [ ]:
        df = pd.DataFrame(doc, columns=['text'])
        df['label'] = label
        #df['file_name'] = re.findall('.+\/(.+\.txt)', f)
        #df = df[['label', 'text']]
        df_all = pd.concat([df_all,df], ignore_index = True)
    return df_all

In [31]:
print(df_en2)

In [25]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
#data_cv = cv.fit_transform(df_en.transcript)
#data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
#data_dtm.index = df_en.index
#data_dtm